In [5]:
import pandas as pd
import numpy as np
import random
from IPython.display import display, Markdown

file_name = "curah_hujan_di_kota_bandung.csv"
random.seed(42)

# Mapping nama bulan -> nomor bulan
bulan_map = {
    "JANUARI": 1, "FEBRUARI": 2, "MARET": 3, "APRIL": 4,
    "MEI": 5, "JUNI": 6, "JULI": 7, "AGUSTUS": 8,
    "SEPTEMBER": 9, "OKTOBER": 10, "NOVEMBER": 11, "DESEMBER": 12
}

def days_in_month(year, month):
    """Jumlah hari per bulan (memperhatikan leap year)."""
    if month in [1,3,5,7,8,10,12]:
        return 31
    if month in [4,6,9,11]:
        return 30
    # Februari
    leap = (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0)
    return 29 if leap else 28


# =====================================================
# 1. MUAT & VALIDASI DATASET
# =====================================================
df = pd.read_csv(file_name)

# Validasi kolom wajib
required_cols = {"bps_nama_kabupaten_kota", "tahun", "bulan", "jumlah_curah_hujan"}
missing_cols = required_cols - set(df.columns)
if missing_cols:
    raise ValueError(f"Kolom berikut tidak ditemukan di dataset: {missing_cols}")

# Filter hanya Kota Bandung
bandung_df = df[df["bps_nama_kabupaten_kota"].str.contains("KOTA BANDUNG", case=False, na=False)].copy()

if bandung_df.empty:
    raise ValueError("Data untuk KOTA BANDUNG tidak ditemukan di dataset.")

# Normalisasi nama bulan agar tahan typo spasi / huruf kecil besar
bandung_df["bulan_upper"] = bandung_df["bulan"].astype(str).str.upper().str.strip()
bandung_df["bulan_num"] = bandung_df["bulan_upper"].map(bulan_map)

# Cek kalau ada bulan yang tidak dikenali
if bandung_df["bulan_num"].isna().any():
    unknown_months = bandung_df.loc[bandung_df["bulan_num"].isna(), "bulan"].unique()
    raise ValueError(f"Ada nama bulan tidak dikenali: {unknown_months}")

# Buang data curah hujan kosong (jaga-jaga)
bandung_df = bandung_df.dropna(subset=["jumlah_curah_hujan"])

# Pastikan tipe numerik
bandung_df["jumlah_curah_hujan"] = pd.to_numeric(bandung_df["jumlah_curah_hujan"], errors="coerce")
bandung_df = bandung_df.dropna(subset=["jumlah_curah_hujan"])

# Sort agar rapi
bandung_df = bandung_df.sort_values(["tahun", "bulan_num"]).reset_index(drop=True)

display(Markdown("## 1. Data Historis Curah Hujan Bulanan Kota Bandung"))
display(bandung_df[["tahun", "bulan", "jumlah_curah_hujan"]].head(12))


## 1. Data Historis Curah Hujan Bulanan Kota Bandung

,tahun,bulan,jumlah_curah_hujan
0,2017,JANUARI,65.3
1,2017,FEBRUARI,199.3
2,2017,MARET,389.3
3,2017,APRIL,220.2
4,2017,MEI,222.3
5,2017,JUNI,106.4
6,2017,JULI,39.1
7,2017,AGUSTUS,48.4
8,2017,SEPTEMBER,90.8
9,2017,OKTOBER,345.3


In [7]:
# =====================================================
# 2. CEK KELENGKAPAN BULAN TIAP TAHUN
# =====================================================
bulan_per_tahun = bandung_df.groupby("tahun")["bulan_num"].nunique()
tahun_tidak_lengkap = bulan_per_tahun[bulan_per_tahun < 12]

if not tahun_tidak_lengkap.empty:
    display(Markdown("❗ **Peringatan:** Ada tahun dengan data bulan tidak lengkap. "
                     "Prediksi tetap jalan, tapi hasil bisa bias."))
    display(tahun_tidak_lengkap)
    
# =====================================================
# 3. BANGUN DISTRIBUSI PROBABILITAS MONTE CARLO PER BULAN
# =====================================================
interval_per_bulan = {}

for m in range(1, 13):
    sub = bandung_df[bandung_df["bulan_num"] == m].copy()
    sub = sub[["tahun", "bulan_upper", "jumlah_curah_hujan"]].reset_index(drop=True)

    # kalau ternyata ada bulan yang tidak punya histori
    if sub.empty:
        raise ValueError(f"Tidak ada data historis untuk bulan ke-{m}.")

    total_bulan = sub["jumlah_curah_hujan"].sum()

    # Probabilitas proporsional terhadap besaran hujan historis bulan itu
    sub["Probabilitas"] = sub["jumlah_curah_hujan"] / total_bulan
    sub["Kumulatif"] = sub["Probabilitas"].cumsum()

    # Interval 0–999
    sub["Batas Bawah"] = (sub["Kumulatif"].shift(1, fill_value=0) * 1000).round().astype(int)
    sub["Batas Atas"] = (sub["Kumulatif"] * 1000).round().astype(int) - 1

    # rapikan ujung interval
    sub.loc[sub.index[0], "Batas Bawah"] = 0
    sub.loc[sub.index[-1], "Batas Atas"] = 999

    sub["Interval"] = sub.apply(lambda r: f"{r['Batas Bawah']} - {r['Batas Atas']}", axis=1)

    interval_per_bulan[m] = sub

display(Markdown("## 2. Contoh Tabel Interval Monte Carlo (Bulan Januari)"))
display(interval_per_bulan[1][["tahun", "bulan_upper", "jumlah_curah_hujan",
                               "Probabilitas", "Kumulatif", "Interval"]])

## 2. Contoh Tabel Interval Monte Carlo (Bulan Januari)

,tahun,bulan_upper,jumlah_curah_hujan,Probabilitas,Kumulatif,Interval
0,2017,JANUARI,65.3,0.067431,0.067431,0 - 66
1,2018,JANUARI,191.0,0.197233,0.264663,67 - 264
2,2019,JANUARI,231.6,0.239157,0.503821,265 - 503
3,2020,JANUARI,207.6,0.214374,0.718195,504 - 717
4,2021,JANUARI,146.4,0.151177,0.869372,718 - 868
5,2022,JANUARI,59.5,0.061442,0.930814,869 - 930
6,2023,JANUARI,67.0,0.069186,1.000000,931 - 999


In [8]:
# =====================================================
# 4. FUNGSI PEMETAAN ANGKA ACAK → CURAH HUJAN BULANAN
# =====================================================
def get_prediction(rand_num, df_interval):
    row = df_interval[
        (df_interval["Batas Bawah"] <= rand_num) &
        (df_interval["Batas Atas"] >= rand_num)
    ]
    if not row.empty:
        return row["jumlah_curah_hujan"].iloc[0]
    return np.nan


# =====================================================
# 5. SIMULASI MONTE CARLO UNTUK TAHUN BERIKUTNYA (BULANAN)
# =====================================================
tahun_target = int(bandung_df["tahun"].max()) + 1

prediksi_bulanan = []
for m in range(1, 13):
    rn = random.randint(0, 999)
    pred_mm = get_prediction(rn, interval_per_bulan[m])

    prediksi_bulanan.append({
        "Tahun Prediksi": tahun_target,
        "Bulan": m,
        "Angka Acak": rn,
        "Prediksi Curah Hujan Bulanan (mm)": float(pred_mm)
    })

prediksi_bulanan_df = pd.DataFrame(prediksi_bulanan)

display(Markdown(f"## 3. Prediksi Curah Hujan Bulanan Tahun {tahun_target} (Monte Carlo)"))
display(prediksi_bulanan_df)

## 3. Prediksi Curah Hujan Bulanan Tahun 2024 (Monte Carlo)

,Tahun Prediksi,Bulan,Angka Acak,Prediksi Curah Hujan Bulanan (mm)
0,2024,1,654,207.6
1,2024,2,114,199.3
2,2024,3,25,389.3
3,2024,4,759,336.2
4,2024,5,281,245.7
5,2024,6,250,33.4
6,2024,7,228,63.7
7,2024,8,142,48.4
8,2024,9,754,773.0
9,2024,10,104,345.3


In [9]:

# =====================================================
# 6. KONVERSI KE PREDIKSI HARIAN PER TANGGAL
# =====================================================
daily_rows = []

for _, row in prediksi_bulanan_df.iterrows():
    m = int(row["Bulan"])
    monthly_mm = row["Prediksi Curah Hujan Bulanan (mm)"]

    hari = days_in_month(tahun_target, m)
    daily_mm = monthly_mm / hari

    dates = pd.date_range(
        start=f"{tahun_target}-{m:02d}-01",
        periods=hari,
        freq="D"
    )

    for d in dates:
        daily_rows.append({
            "Tanggal": d,
            "Prediksi Curah Hujan Harian (mm/hari)": daily_mm
        })

prediksi_harian_df = pd.DataFrame(daily_rows)

display(Markdown(f"## 4. Prediksi Curah Hujan Harian Kota Bandung Tahun {tahun_target}"))
display(prediksi_harian_df.head(31))  # tampilkan 1 bulan pertama saja


# =====================================================
# 7. RINGKASAN HASIL
# =====================================================
total_tahun_prediksi = prediksi_bulanan_df["Prediksi Curah Hujan Bulanan (mm)"].sum()
rata_harian_tahun_prediksi = prediksi_harian_df["Prediksi Curah Hujan Harian (mm/hari)"].mean()

display(Markdown(f"""
---
## Ringkasan Prediksi Tahun {tahun_target}

- **Total curah hujan tahunan prediksi:** **{total_tahun_prediksi:.2f} mm/tahun**
- **Rata-rata curah hujan harian prediksi:** **{rata_harian_tahun_prediksi:.2f} mm/hari**

Output utama studi kasus sudah berupa **prediksi harian per tanggal**.
---
"""))

## 4. Prediksi Curah Hujan Harian Kota Bandung Tahun 2024

,Tanggal,Prediksi Curah Hujan Harian (mm/hari)
0,2024-01-01,6.696774
1,2024-01-02,6.696774
2,2024-01-03,6.696774
3,2024-01-04,6.696774
4,2024-01-05,6.696774
5,2024-01-06,6.696774
6,2024-01-07,6.696774
7,2024-01-08,6.696774
8,2024-01-09,6.696774
9,2024-01-10,6.696774



---
## Ringkasan Prediksi Tahun 2024

- **Total curah hujan tahunan prediksi:** **5081.20 mm/tahun**
- **Rata-rata curah hujan harian prediksi:** **13.88 mm/hari**

Output utama studi kasus sudah berupa **prediksi harian per tanggal**.
---
